[Open in Colab](https://colab.research.google.com/drive/1AJ-r9IfJOrLTzpA5p68WYFphSDk0TOUO)


In [1]:
from dotenv import load_dotenv
import os
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

# Step 1: User inputs a contract address (and chain location)

In [3]:
CONTRACT_ADDRESS = '0x2416092f143378750bb29b79ed961ab195cceea5'
CHAIN_NAME = 'OPTIMISM'

# Step 2: Check if the contract is associated with any known projects in OSO or Atlas

In [7]:
result = client.to_pandas(f"""
  SELECT
    abp.artifact_id,
    p.project_source,
    p.project_id,
    p.project_name,
    p.display_name
  FROM artifacts_by_project_v1 AS abp
  JOIN projects_v1 AS p ON abp.project_id = p.project_id
  WHERE
    abp.artifact_name = LOWER('{CONTRACT_ADDRESS}')
    AND abp.artifact_source = UPPER('{CHAIN_NAME}')
""")
result

,artifact_id,project_source,project_id,project_name,display_name
0,R/4j0CBAMP7Y6IWPuKfXET2QZWa8hgQ2I/FN+XQd2UI=,OSS_DIRECTORY,8cby1aMn7r59alz/BdKfcrjF0m4W+I2AYffy8Ac85Iw=,renzo-protocol,Renzo Protocol


In [14]:
if not result.empty:
    ARTIFACT_ID = result['artifact_id'].values[0]
ARTIFACT_ID    

'R/4j0CBAMP7Y6IWPuKfXET2QZWa8hgQ2I/FN+XQd2UI='

# Step 3: Check if the contract has sufficient activity to qualify for the onchain builder round

Warning: this is current a VERY LARGE table and may take a few minutes to process.

In [16]:
START_DATE = '2024-12-01'
END_DATE = '2025-06-01'

In [19]:
metrics = client.to_pandas(f"""
  SELECT
    SUM(tm.amount) AS transaction_count,
    COUNT(DISTINCT tm.sample_date) AS active_days
  FROM timeseries_metrics_by_artifact_v0 AS tm
  JOIN metrics_v0 AS m ON tm.metric_id = m.metric_id
  WHERE
    m.metric_name = '{CHAIN_NAME}_contract_invocations_daily'
    AND tm.artifact_id = '{ARTIFACT_ID}'
    AND tm.sample_date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
""")
metrics

,transaction_count,active_days
0,209705,178


In [26]:
TRANSACTION_THRESHOLD = 1000
ACTIVE_DAYS_THRESHOLD = 10
if metrics['transaction_count'].max() >= TRANSACTION_THRESHOLD and metrics['active_days'].max() >= ACTIVE_DAYS_THRESHOLD:
    print("Eligible")
else:
    print("Not eligible")

Eligible


# Step 4: Look up expected deployer

In [32]:
contract = client.to_pandas(f"""

  SELECT *
  FROM contracts_v0
  WHERE
    contract_address = '{CONTRACT_ADDRESS}'
    AND contract_namespace = '{CHAIN_NAME}'

""")
contract.T

,0
deployment_date,2024-06-14
contract_address,0x2416092f143378750bb29b79ed961ab195cceea5
contract_namespace,OPTIMISM
originating_address,0xadef586efb3287da4d7d1cbe15f12e0be69e0df0
factory_address,0xf30cec7686d8b0015bbcbfe55f79dd75d0302ec5
root_deployer_address,0xadef586efb3287da4d7d1cbe15f12e0be69e0df0
sort_weight,560908


In [31]:
if not contract.empty:
    print("Project can claim this contract by verifying ownership of deployer address", contract['root_deployer_address'].values[0])

Project can claim this contract by verifying ownership of deployer address 0xadef586efb3287da4d7d1cbe15f12e0be69e0df0
